In [50]:
import pandas
import statsmodels.api as sm

# Load Dataset
The 'sample_data.csv' is from Rdatasets. A description of the data can be found here:  
https://vincentarelbundock.github.io/Rdatasets/doc/Stat2Data/FishEggs.html

In [51]:
filename = 'sample_data.csv'

data = pandas.read_csv(filename)
data.head()

,Unnamed: 0,Age,PctDM,Month,Sept
0,0,7,37.35,Nov,0
1,1,8,38.05,Nov,0
2,2,8,37.45,Nov,0
3,3,9,38.95,Nov,0
4,4,9,37.90,Nov,0


In [17]:
outcome = 'PctDM'
predictors = ['Age', 'Sept']

In [39]:
formula = outcome + '~' + predictors[0] + '+' + predictors[1]
print(formula)
fit = sm.OLS.from_formula(formula, data).fit()
fit.summary()

PctDM~Age+Sept


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  PctDM   R-squared:                       0.430
Model:                            OLS   Adj. R-squared:                  0.394
Method:                 Least Squares   F-statistic:                     12.06
Date:                Wed, 13 Oct 2021   Prob (F-statistic):           0.000125
Time:                        16:32:11   Log-Likelihood:                -55.144
No. Observations:                  35   AIC:                             116.3
Df Residuals:                      32   BIC:                             121.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     39.5192      0.778     50.778      0.000      37.934      41.105
Age           -0.2287      0.063     -3.635      0.001      -0.357      -0.101
Sept          -1.5193      0.423     -3.588      0.001      -2.382      -0.657
==============================================================================
Omnibus:                        1.371   Durbin-Watson:                   2.485
Prob(Omnibus):                  0.504   Jarque-Bera (JB):                1.067
Skew:                          -0.421   Prob(JB):                        0.587
Kurtosis:                       2.848   Cond. No.                         45.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [36]:
fit.pvalues[0]

3.660265387975013e-32

In [49]:
#predictors_remaining = predictors.copy()
predictors_remaining = ['Age']
predictors_kept = 'Sept' + '+'

pvalues = {predictors[0]: None, predictors[1]: None}
# TODO: turn this into dataframe for different levels

for predictor in predictors_remaining:
    formula = outcome + '~' + predictors_kept + predictor
    fit = sm.OLS.from_formula(formula, data).fit()
    pvalues[predictor] = fit.pvalues[1]
    # print(fit.summary())

pvalues

{'Age': 0.0010962525075049604, 'Sept': None}

In [45]:
min_pvalue = min(pvalues.values())
min_pvalue

0.007001273350771227

In [46]:
list(pvalues.keys())[list(pvalues.values()).index(min_pvalue)]

'Age'

In [ ]:
def find_important_variables(data: pandas.DataFrame, outcome: str, predictors: list, evaluation_measure='pvalue', evaluation_threshold=0.05):
    predictors_remaining = predictors
    predictors_kept = ''
    num_predictors_kept = 0

    pvalues = pandas.DataFrame(columns=predictors, index=range(len(predictors)))

    for predictor in predictors_remaining:
        # Create function formula with one additional predictor
        formula = outcome + '~' + predictors_kept + predictor
        # Fit a linear model with the above formula
        model_fit = sm.OLS.from_formula(formula, data).fit()
        # Record the pvalue of the new predictor
        pvalues.loc[num_predictors_kept, predictor] = model_fit.pvalues[num_predictors_kept + 1]